In [4]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage.feature import hog
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
%matplotlib inline


In [5]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [6]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,35,55,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female  male
age                   
(9, 19]     3870  4350
(19, 29]    5760  7590
(29, 39]    1530  2580
(39, 49]    2070  1890
(49, 59]    1590  1170
(59, 69]     420   450
(69, 79]     360    30


In [22]:
def label_fetch(age):
    if age >= 10 and age <= 35:
        return ("18-35")
    elif age > 35 and age <= 55:
        return ("36-55")
    elif age > 55:
        return ("greater than 55")

In [26]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
hog_features = []
age_label = []
male_age_label = []
female_age_label = []
male_hog_features = []
female_hog_features = []


with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #Resizing the images
        resized_img = resize(img, (128,64)) 
        #creating hog features for the data
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)

        #Set age range label after gender bias
        #appending the hog features for image onto the hog data lists 
        hog_features.append(fd)
        age_label.append(label_fetch(age[i]))
        if gender[i]== 'male':
            male_hog_features.append(fd)
            male_age_label.append(label_fetch(age[i]))
        else:
            female_hog_features.append(fd)
            female_age_label.append(label_fetch(age[i]))
                            

In [27]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(male_age_label)
X =  np.array(male_hog_features)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

#Linear SVM
print("MALE")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

MALE
Overall Accuracy using Linear SVM: 0.8976651763537009
                 precision    recall  f1-score   support

          18-35       0.89      0.99      0.94      1387
          36-55       0.92      0.71      0.80       542
greater than 55       0.96      0.63      0.76        84

       accuracy                           0.90      2013
      macro avg       0.93      0.78      0.83      2013
   weighted avg       0.90      0.90      0.89      2013

Accuracy for each class
[0.98846431 0.70664207 0.63095238]




Non-Linear RBF Accuracy: 0.6890213611525087
                 precision    recall  f1-score   support

          18-35       0.69      1.00      0.82      1387
          36-55       0.00      0.00      0.00       542
greater than 55       0.00      0.00      0.00        84

       accuracy                           0.69      2013
      macro avg       0.23      0.33      0.27      2013
   weighted avg       0.47      0.69      0.56      2013

                 precision    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.9458519622454049
                 precision    recall  f1-score   support

          18-35       0.95      0.98      0.97      1387
          36-55       0.93      0.87      0.90       542
greater than 55       0.96      0.83      0.89        84

       accuracy                           0.95      2013
      macro avg       0.95      0.90      0.92      2013
   weighted avg       0.95      0.95      0.95      2013

                 precision    recall  f1-score   support

          18-35       0.95      0.98      0.97      1387
          36-55       0.93      0.87      0.90       542
greater than 55       0.96      0.83      0.89        84

       accuracy                           0.95      2013
      macro avg       0.95      0.90      0.92      2013
   weighted avg       0.95      0.95      0.95      2013

Accuracy for each class
[0.98053353 0.87453875 0.83333333]




Iteration 1, loss = 0.99095132
Iteration 2, loss = 0.77144838
Iteration 3, loss = 0.7556

Iteration 225, loss = 0.02142291
Iteration 226, loss = 0.02076937
Iteration 227, loss = 0.02054858
Iteration 228, loss = 0.02036957
Iteration 229, loss = 0.02004248
Iteration 230, loss = 0.01959160
Iteration 231, loss = 0.01924060
Iteration 232, loss = 0.01906714
Iteration 233, loss = 0.01888623
Iteration 234, loss = 0.01843236
Iteration 235, loss = 0.01823559
Iteration 236, loss = 0.01792189
Iteration 237, loss = 0.01754447
Iteration 238, loss = 0.01755257
Iteration 239, loss = 0.01706846
Iteration 240, loss = 0.01687970
Iteration 241, loss = 0.01670840
Iteration 242, loss = 0.01641611
Iteration 243, loss = 0.01618877
Iteration 244, loss = 0.01607026
Iteration 245, loss = 0.01580539
Iteration 246, loss = 0.01551633
Iteration 247, loss = 0.01527414
Iteration 248, loss = 0.01522046
Iteration 249, loss = 0.01510192
Iteration 250, loss = 0.01484425
Iteration 251, loss = 0.01447973
Iteration 252, loss = 0.01423307
Iteration 253, loss = 0.01405877
Iteration 254, loss = 0.01389903
Iteration 

Iteration 474, loss = 0.00271445
Iteration 475, loss = 0.00269300
Iteration 476, loss = 0.00268285
Iteration 477, loss = 0.00267044
Iteration 478, loss = 0.00266802
Iteration 479, loss = 0.00264647
Iteration 480, loss = 0.00263782
Iteration 481, loss = 0.00262549
Iteration 482, loss = 0.00261158
Iteration 483, loss = 0.00260043
Iteration 484, loss = 0.00259559
Iteration 485, loss = 0.00258257
Iteration 486, loss = 0.00257712
Iteration 487, loss = 0.00255670
Iteration 488, loss = 0.00254799
Iteration 489, loss = 0.00253553
Iteration 490, loss = 0.00252940
Iteration 491, loss = 0.00251700
Iteration 492, loss = 0.00251049
Iteration 493, loss = 0.00249759
Iteration 494, loss = 0.00249667
Iteration 495, loss = 0.00247922
Iteration 496, loss = 0.00248315
Iteration 497, loss = 0.00246068
Iteration 498, loss = 0.00244911
Iteration 499, loss = 0.00243539
Iteration 500, loss = 0.00242784
MLP Accuracy: 0.8777943368107303
                 precision    recall  f1-score   support

          18-35   

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [28]:
# Fit a SVM classifier to the data where gender=female
y =  np.array(female_age_label)
X = np.array(female_hog_features)   
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("FEMALE")
print("Overall Accuracy using Linear SVM:"+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0
    
#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
    
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')   
#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
    
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
    
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')


FEMALE
Overall Accuracy using Linear SVM:0.9347319347319347
                 precision    recall  f1-score   support

          18-35       0.93      1.00      0.96      1030
          36-55       0.97      0.66      0.78       201
greater than 55       1.00      0.77      0.87        56

       accuracy                           0.93      1287
      macro avg       0.97      0.81      0.87      1287
   weighted avg       0.94      0.93      0.93      1287

Accuracy for each class
[0.99805825 0.65671642 0.76785714]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

                 precision    recall  f1-score   support

          18-35       0.80      1.00      0.89      1030
          36-55       0.00      0.00      0.00       201
greater than 55       0.00      0.00      0.00        56

       accuracy                           0.80      1287
      macro avg       0.27      0.33      0.30      1287
   weighted avg       0.64      0.80      0.71      1287

Accuracy for each class
[1. 0. 0.]




Non-Linear RBF Accuracy: 0.8003108003108003
                 precision    recall  f1-score   support

          18-35       0.80      1.00      0.89      1030
          36-55       0.00      0.00      0.00       201
greater than 55       0.00      0.00      0.00        56

       accuracy                           0.80      1287
      macro avg       0.27      0.33      0.30      1287
   weighted avg       0.64      0.80      0.71      1287

Accuracy for each class
[1. 0. 0.]




Non-Linear Poly Accuracy: 0.9696969696969697
                 precision   

Iteration 210, loss = 0.03517800
Iteration 211, loss = 0.03446128
Iteration 212, loss = 0.03390772
Iteration 213, loss = 0.03359268
Iteration 214, loss = 0.03299799
Iteration 215, loss = 0.03251374
Iteration 216, loss = 0.03199325
Iteration 217, loss = 0.03152685
Iteration 218, loss = 0.03113713
Iteration 219, loss = 0.03131233
Iteration 220, loss = 0.03048117
Iteration 221, loss = 0.02984691
Iteration 222, loss = 0.02950258
Iteration 223, loss = 0.02905362
Iteration 224, loss = 0.02894098
Iteration 225, loss = 0.02830885
Iteration 226, loss = 0.02781382
Iteration 227, loss = 0.02754053
Iteration 228, loss = 0.02699279
Iteration 229, loss = 0.02703136
Iteration 230, loss = 0.02708610
Iteration 231, loss = 0.02616403
Iteration 232, loss = 0.02596297
Iteration 233, loss = 0.02553091
Iteration 234, loss = 0.02499913
Iteration 235, loss = 0.02479433
Iteration 236, loss = 0.02459851
Iteration 237, loss = 0.02429620
Iteration 238, loss = 0.02413955
Iteration 239, loss = 0.02364371
Iteration 

Iteration 460, loss = 0.00397384
Iteration 461, loss = 0.00395378
Iteration 462, loss = 0.00395741
Iteration 463, loss = 0.00393450
Iteration 464, loss = 0.00390271
Iteration 465, loss = 0.00403686
Iteration 466, loss = 0.00391451
Iteration 467, loss = 0.00383808
Iteration 468, loss = 0.00385313
Iteration 469, loss = 0.00390324
Iteration 470, loss = 0.00381929
Iteration 471, loss = 0.00378358
Iteration 472, loss = 0.00387008
Iteration 473, loss = 0.00374957
Iteration 474, loss = 0.00378002
Iteration 475, loss = 0.00371209
Iteration 476, loss = 0.00376939
Iteration 477, loss = 0.00367829
Iteration 478, loss = 0.00367656
Iteration 479, loss = 0.00365387
Iteration 480, loss = 0.00361729
Iteration 481, loss = 0.00362748
Iteration 482, loss = 0.00365585
Iteration 483, loss = 0.00358503
Iteration 484, loss = 0.00356823
Iteration 485, loss = 0.00354915
Iteration 486, loss = 0.00353322
Iteration 487, loss = 0.00350471
Iteration 488, loss = 0.00349931
Iteration 489, loss = 0.00347668
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label)
X =  np.array(hog_features)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

#Linear SVM
print("Overall")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

MALE
Overall Accuracy using Linear SVM: 0.8996969696969697
                 precision    recall  f1-score   support

          18-35       0.90      0.99      0.94      2408
          36-55       0.90      0.67      0.77       740
greater than 55       0.99      0.59      0.74       152

       accuracy                           0.90      3300
      macro avg       0.93      0.75      0.82      3300
   weighted avg       0.90      0.90      0.89      3300

Accuracy for each class
[0.9904485  0.66756757 0.59210526]




Non-Linear RBF Accuracy: 0.7296969696969697
                 precision    recall  f1-score   support

          18-35       0.73      1.00      0.84      2408
          36-55       0.00      0.00      0.00       740
greater than 55       0.00      0.00      0.00       152

       accuracy                           0.73      3300
      macro avg       0.24      0.33      0.28      3300
   weighted avg       0.53      0.73      0.62      3300

                 precision    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.9418181818181818
                 precision    recall  f1-score   support

          18-35       0.95      0.98      0.96      2408
          36-55       0.92      0.83      0.87       740
greater than 55       0.95      0.88      0.91       152

       accuracy                           0.94      3300
      macro avg       0.94      0.89      0.92      3300
   weighted avg       0.94      0.94      0.94      3300

                 precision    recall  f1-score   support

          18-35       0.95      0.98      0.96      2408
          36-55       0.92      0.83      0.87       740
greater than 55       0.95      0.88      0.91       152

       accuracy                           0.94      3300
      macro avg       0.94      0.89      0.92      3300
   weighted avg       0.94      0.94      0.94      3300

Accuracy for each class
[0.98131229 0.82702703 0.875     ]




Iteration 1, loss = 0.87596820
Iteration 2, loss = 0.70236369
Iteration 3, loss = 0.6966

Iteration 225, loss = 0.01156819
Iteration 226, loss = 0.01145504
Iteration 227, loss = 0.01126032
Iteration 228, loss = 0.01106352
Iteration 229, loss = 0.01088698
Iteration 230, loss = 0.01067172
Iteration 231, loss = 0.01067342
Iteration 232, loss = 0.01041061
Iteration 233, loss = 0.01027803
Iteration 234, loss = 0.01019875
Iteration 235, loss = 0.00994555
Iteration 236, loss = 0.00986879
Iteration 237, loss = 0.00969535
Iteration 238, loss = 0.00951318
Iteration 239, loss = 0.00939624
Iteration 240, loss = 0.00926905
Iteration 241, loss = 0.00920134
Iteration 242, loss = 0.00902044
Iteration 243, loss = 0.00889872
Iteration 244, loss = 0.00875680
Iteration 245, loss = 0.00870728
Iteration 246, loss = 0.00856457
Iteration 247, loss = 0.00842129
Iteration 248, loss = 0.00836093
Iteration 249, loss = 0.00826255
Iteration 250, loss = 0.00811816
Iteration 251, loss = 0.00801643
Iteration 252, loss = 0.00791349
Iteration 253, loss = 0.00782783
Iteration 254, loss = 0.00774951
Iteration 

Iteration 474, loss = 0.00167958
Iteration 475, loss = 0.00167346
Iteration 476, loss = 0.00166926
Iteration 477, loss = 0.00166660
Iteration 478, loss = 0.00165275
Iteration 479, loss = 0.00164858
Iteration 480, loss = 0.00164099
Iteration 481, loss = 0.00164283
Iteration 482, loss = 0.00163157
Iteration 483, loss = 0.00162379
Iteration 484, loss = 0.00161484
Iteration 485, loss = 0.00161050
Iteration 486, loss = 0.00160390
Iteration 487, loss = 0.00160303
Iteration 488, loss = 0.00159064
Iteration 489, loss = 0.00158306
Iteration 490, loss = 0.00158076
Iteration 491, loss = 0.00157363
Iteration 492, loss = 0.00156557
Iteration 493, loss = 0.00156667
Iteration 494, loss = 0.00156376
Iteration 495, loss = 0.00154717
Iteration 496, loss = 0.00154578
Iteration 497, loss = 0.00154099
Iteration 498, loss = 0.00153645
Iteration 499, loss = 0.00152711
Iteration 500, loss = 0.00152302
MLP Accuracy: 0.8872727272727273
                 precision    recall  f1-score   support

          18-35   

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
